In [9]:
from rake_nltk import Rake
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('EdTech.courses.csv')
df.head()

,_id,name,description,thumbnail,status,price,hasExam,owner,createdAt,updatedAt,__v
0,66e2b091932333a9f737aa76,Complete React course for beginner,Let's learn react,https://i.ytimg.com/vi/eCU7FfMl5WU/default.jpg,published,99,False,66b5a6c2cd3c10208dfc5289,2024-09-12T09:12:49.480Z,2024-09-12T09:12:49.480Z,0
1,66e2db94ce0d26e4e8b3f1f9,Backend development in JavaScript,I just love Ice tea and I love code. This seri...,https://i.ytimg.com/vi/nhmMx-ys_zQ/default.jpg,published,99,False,66b5a6c2cd3c10208dfc5289,2024-09-12T12:16:20.917Z,2024-09-12T12:25:34.707Z,0
2,66e2dd65ce0d26e4e8b3f362,MongoDB aggregation Pipelines,Learn mongoDB aggregation pipelines. If you ar...,https://i.ytimg.com/vi/SUZKhBvxW5c/default.jpg,published,99,False,66b5a6c2cd3c10208dfc5289,2024-09-12T12:24:05.099Z,2024-09-12T12:24:05.099Z,0
3,66e2de54ce0d26e4e8b3f419,Git for beginner,Git for beginner is a series focused on folks ...,https://i.ytimg.com/vi/7tOLcNZfPso/default.jpg,published,99,False,66b5a6c2cd3c10208dfc5289,2024-09-12T12:28:04.720Z,2024-09-12T12:28:04.720Z,0
4,66e2dfe7ce0d26e4e8b3f4ce,Javascript course,A Good JavaScript Series,https://i.ytimg.com/vi/2md4HQNRqJA/default.jpg,published,99,False,66b5a6c2cd3c10208dfc5289,2024-09-12T12:34:47.422Z,2024-09-12T12:36:35.025Z,0


In [3]:
df.shape

(16, 11)

In [4]:
df = df[['_id','name','description']]
df.head()

,_id,name,description
0,66e2b091932333a9f737aa76,Complete React course for beginner,Let's learn react
1,66e2db94ce0d26e4e8b3f1f9,Backend development in JavaScript,I just love Ice tea and I love code. This seri...
2,66e2dd65ce0d26e4e8b3f362,MongoDB aggregation Pipelines,Learn mongoDB aggregation pipelines. If you ar...
3,66e2de54ce0d26e4e8b3f419,Git for beginner,Git for beginner is a series focused on folks ...
4,66e2dfe7ce0d26e4e8b3f4ce,Javascript course,A Good JavaScript Series


In [5]:
df.isnull()

,_id,name,description
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
7,False,False,False
8,False,False,False
9,False,False,True


In [6]:
df['description'] = df['description'].fillna('')

In [7]:
df['combined'] = df['name'] + " " + df['description']
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined'])

def generate_recommendations(query, top_n=3):
    query_tfidf = tfidf_vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    top_n_indices = cosine_similarities.argsort()[-top_n:][::-1]
    recommended_videos = df.iloc[top_n_indices][['_id','name']]
    return recommended_videos

class RecommendationModel:
    def __init__(self, vectorizer, matrix, data):
        self.vectorizer = vectorizer
        self.matrix = matrix
        self.data = data
    
    # The recommendation function needs access to the vectorizer and matrix
    def generate_recommendations(self, query, top_n=3):
        query_tfidf = self.vectorizer.transform([query])
        cosine_similarities = cosine_similarity(query_tfidf, self.matrix).flatten()
        top_n_indices = cosine_similarities.argsort()[-top_n:][::-1]
        return self.data.iloc[top_n_indices][['_id','title']]

# Create an instance of the model with vectorizer, matrix, and data
course_recommendation_model = RecommendationModel(tfidf_vectorizer, tfidf_matrix, df)

In [10]:
with open('course_recommendation_model.pkl', 'wb') as file:
    pickle.dump(course_recommendation_model, file)

In [ ]:
generate_recommendations("react")